## Script to produce webpages of schedules, abstracts, and authors index from coordinated databases of presentations of SMPC 2019

In [1]:
import collections
import os
import shutil
import io  
import sys  
import urllib
import csv

from time import strptime
from datetime import datetime
from datetime import date

from dumptruck import DumpTruck
from collections import OrderedDict
from collections import Counter
from collections import deque
from os import listdir, chdir,makedirs
import string

# Functions to process entries

In [2]:
def InstSort(InstitutionList):
    #find unique Institutions
    InstList = []
    for inst in InstitutionList:
        yesAdd = 1
        for est in InstList:
            if inst == est:
                yesAdd = 0
        if yesAdd==1:
            InstList.append(inst)
    
    InstIndex = []
    for inst in InstitutionList:
        for k in range(len(InstList)):
            if InstList[k] == inst:
                InstIndex.append(k+1)
    
    InstitutionsSorted = [InstIndex, InstList]
    return InstitutionsSorted

In [3]:
# function producing code for entry with abstract
def talkHTMLAbstract(talk):
    talkCode = (u'%s-%i' % (talk['SessionCode'],talk['TalkSessionNumber']))
    Title = (u'<H3> <a id="%s">%s: %s</H3>\n' % 
                    (talkCode,talkCode, talk['TalkTitle']))
    tDetails =Title

    Is = InstSort(talk['Institutions'])
    InstIndex = Is[0]
    InstList = Is[1]
    
    authors =u''
    for i in range(len(talk['Authors'])):
        loc = u'./author_index.html#' + urllib.quote(talk['AuthorIndex'][i].encode('ascii',errors='ignore'))
        authors += '<a href="'+loc+'">' + talk['Authors'][i] + '</a>' # Author
        authors += '(' + str(InstIndex[i]) + ')'# Institution affiliations
        if i < len(talk['Authors'])-1:
            authors += ', '
    authors += '<br>\n'
    instit =  u'<i> 1:' + Is[1][0]
    for i in range(len(Is[1])-1):
        instit += ', ' + str(i+2) + ':' + Is[1][i+1]
    instit += ' </i> \n <br>'
    tDetails += authors + instit
    tDetails += '<p>'
    abstract = u''
    if talk['Abstract']:
        abstract += talk['Abstract'] + '<p>\n\n'
    #cont = (u'Contact: %s <br>' % (talk['Contact']))
    subjects = u'<b>Subjects:</b> '
    for k in talk['Subjects']:
        subjects += k + ', '
    subjects = subjects[:-2] + '\n<p>\n'
    when = (u'<b>When: %s in %s on %s </b> \n <br>' % 
                    (str(talk['Time']),talk['Room'],talk['Date'] ))
    # Anchor for reference back from author index
    schedDay = (u'Return to <a href="./schedule.html#talks%s-%s">Day Schedule</a>. <br>' % (talk['Date'][-1:],talk['SessionCode']))
    sched = (u'Return to <a href="./schedule.html#day%s">Full Schedule</a>. <br>' % (talk['Date'][-1:]))
    texTalk = tDetails  + abstract + subjects + when + schedDay + sched
    return texTalk

In [4]:
# full abstract entry formating for posters
def posterHTMLAbstract(poster):
    talkCode = (u'%s-%i' % (poster['PosterSession'],poster['PositionNumber']))
    Title = (u'<H3> <a id="%s">%s: %s</H3>\n' % 
                    (talkCode,talkCode,poster['PosterTitle']))
    tDetails =Title
    Is = InstSort(poster['Institutions'])
    InstIndex = Is[0]
    InstList = Is[1]    
    authors =u''
    for i in range(len(poster['Authors'])):
        loc = u'./author_index.html#' + urllib.quote(poster['AuthorIndex'][i].encode('ascii',errors='ignore'))
        authors += '<a href="'+loc+'">' + poster['Authors'][i] + '</a>' # Author
        authors += '(' + str(InstIndex[i]) + ')'# Institution affiliations
        if i < len(poster['Authors'])-1:
            authors += ', '
    authors += '<br>\n'
    instit =  u'<i> 1:' + Is[1][0]
    for i in range(len(Is[1])-1):
        instit += ', ' + str(i+2) + ':' + Is[1][i+1]
    instit += ' </i> \n <br>'    
    tDetails += authors + instit
    tDetails += '<p>'
    abstract = u''
    if poster['Abstract']:
        abstract += poster['Abstract'] + '<p>\n\n'
    #cont = (u'Contact: %s <br>' % (talk['Contact']))
    subjects = u'<b>Subjects:</b> '
    for k in poster['Subjects']:
        subjects += k + ', '
    subjects = subjects[:-2] + '\n<p>\n' 
    when = (u'\n<b>When: %s on %s </b> \n <br>' % (poster['Time'],poster['Date'] ))
    schedDay = (u'Return to <a href="./schedule.html#posters%s-%s">Day Schedule</a>. <br>' % (poster['Date'][-1:],poster['PosterSession']))
    sched = (u'Return to <a href="./schedule.html#day%s">Full Schedule</a>. <br>' % (poster['Date'][-1:]))
    htmlPoster = tDetails + abstract + subjects + when + schedDay + sched
    return htmlPoster

In [5]:
# function with formating for talk details without abstract, for session lists.
def talkDets(talk):
    Title = (u'<b>%s-%i: <a href="./abstracts.html#%s-%i">%s</a></b>\n <br>' % 
                    (talk['SessionCode'],talk['TalkSessionNumber'],
                     talk['SessionCode'],talk['TalkSessionNumber'], talk['TalkTitle']))
    tDetails =Title
    #f.write(sessionTitle)
    
    authors =u''
    for i in range(len(talk['Authors'])):
        loc = u'./author_index.html#' + urllib.quote(talk['AuthorIndex'][i].encode('ascii',errors='ignore'))
        authors += '<a href="'+loc+'">' + talk['Authors'][i] + '</a>' # Author
        if i < len(talk['Authors'])-1:
            authors += ', '
    authors += '<br>\n'
    tDetails += authors
    texTalk = tDetails
    return texTalk

In [6]:
# function with formating for presentation details without abstract, for session lists.

def posterDets(talk):
    # poster code
    Title = (u'<b> %s-%s: <a href="./abstracts.html#%s-%s">%s</a></b><br>\n' % 
                    (poster['PosterSession'],str(poster['PositionNumber']),poster['PosterSession'],
                     str(poster['PositionNumber']), poster['PosterTitle']))
    tDetails =Title
    #f.write(sessionTitle)
    authors =u''
    for i in range(len(poster['Authors'])):
        loc = u'./author_index.html#' + urllib.quote(poster['AuthorIndex'][i].encode('ascii',errors='ignore'))
        authors += '<a href="'+loc+'">' + poster['Authors'][i] + '</a>' # Author
        if i < len(poster['Authors'])-1:
            authors += ', '
    authors += '<br>\n'
    tDetails += authors
    texTalk = tDetails
    return texTalk

# product specific pages of html code with internal linking

In [7]:
#HTML for abstracts page, starting with Aug 5 talks 
chdir('./data')
dt = DumpTruck(dbname='Aug5_Talks.db')
ds = DumpTruck(dbname='Aug5_Sessions.db')
data = ds.dump()
chdir('..')
chdir('./Web')

# OK now write to file

auts = [] # components for the authors index

fileName = u'./abstracts.html'
f=io.open(fileName, 'w+', encoding='utf8') # hoping here
f.write(u'<html><head><title>Abstracts</title></head><body>\n')

for i in range(len(data)):
    # session title
    ss = ds.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(i+1))
    sessionHeading = (u'<H2><a id="%s"></a>Session %s, %s </H2>\n %s in %s\n <br>'
                      % (ss[0]['SessionCode'],ss[0]['SessionCode'],ss[0]['SessionTitle'],ss[0]['Times'],ss[0]['Room']))
    f.write(sessionHeading)
    ts = dt.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(i+1))

    for talk in ts:
        talkCode = (u'%s-%i' % (talk['SessionCode'],talk['TalkSessionNumber']))
        auts.append([talkCode,talk['AuthorIndex']])
        anch = (u'<a id="%s">\n' % (talk['TalkSessionNumber']))
        details=talkHTMLAbstract(talk)
        f.write(anch+details)


## WARNING THIS DOES NOT CLOSE THE HTML TAGS OR THE FILE UNTIL ALL DAYS A PROCESSED
chdir('..')
chdir('./data')
dt = DumpTruck(dbname='Aug6_Talks.db')
ds = DumpTruck(dbname='Aug6_Sessions.db')
data = ds.dump()
chdir('..')
chdir('./Web')

# OK now write to file
for i in range(len(data)):
    # session title
    ss = ds.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(i+1))
    sessionHeading = (u'<H2><a id="%s"></a>Session %s, %s </H2>\n %s in %s\n <br>'
                      % (ss[0]['SessionCode'],ss[0]['SessionCode'],ss[0]['SessionTitle'],ss[0]['Times'],ss[0]['Room']))
    f.write(sessionHeading)
    ts = dt.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(i+1))

    for talk in ts:
        talkCode = (u'%s-%i' % (talk['SessionCode'],talk['TalkSessionNumber']))
        auts.append([talkCode,talk['AuthorIndex']])
        anch = (u'<a id="%s">\n' % (talk['TalkSessionNumber']))
        details=talkHTMLAbstract(talk)
        f.write(anch+details)

chdir('..')
chdir('./data')
dt = DumpTruck(dbname='Aug7_Talks.db')
ds = DumpTruck(dbname='Aug7_Sessions.db')
data = ds.dump()
chdir('..')
chdir('./Web')

# OK now write to file
for i in range(len(data)):
    # session title
    ss = ds.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(i+1))
    sessionHeading = (u'<H2><a id="%s"></a>Session %s, %s </H2>\n %s in %s\n <br>'
                      % (ss[0]['SessionCode'],ss[0]['SessionCode'],ss[0]['SessionTitle'],ss[0]['Times'],ss[0]['Room']))
    f.write(sessionHeading)
    ts = dt.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(i+1))

    for talk in ts:
        talkCode = (u'%s-%i' % (talk['SessionCode'],talk['TalkSessionNumber']))
        auts.append([talkCode,talk['AuthorIndex']])
        anch = (u'<a id="%s">\n' % (talk['TalkSessionNumber']))
        details=talkHTMLAbstract(talk)
        f.write(anch+details)

        
chdir('..')
chdir('./data')
dt = DumpTruck(dbname='Aug6_Posters.db')
ds = DumpTruck(dbname='Aug6_Sessions.db')
chdir('..')
chdir('./Web')

# OK now write to file

#auts = [] # components for the authors index

data = ds.dump()

for i in ['1','2']:
    ps = dt.execute('SELECT * FROM `dumptruck` WHERE `PosterSessionNumber` = ' + i)
    sessionHeading = (u'<H2><a id="%s"></a>Poster session %s </H2>\n %s in %s\n <br>'
                      % (ps[0]['PosterSession'],ps[0]['PosterSession'],ps[0]['Time'],ps[0]['Room']))
    f.write(sessionHeading)
    for talk in ps:
        talkCode = (u'%s-%i' % (talk['PosterSession'],talk['PositionNumber']))
        auts.append([talkCode,talk['AuthorIndex']])
        anch = (u'<a id="%s">\n' % (talk['PositionNumber']))
        details=posterHTMLAbstract(talk)
        f.write(anch+details)       
        

        
chdir('..')
chdir('./data')
dt = DumpTruck(dbname='Aug7_Posters.db')
ds = DumpTruck(dbname='Aug7_Sessions.db')
chdir('..')
chdir('./Web')

data = ds.dump()

for i in ['3','4']:
    ps = dt.execute('SELECT * FROM `dumptruck` WHERE `PosterSessionNumber` = ' + i)
    sessionHeading = (u'<H2><a id="%s"></a>Poster session %s </H2>\n %s in %s\n <br>'
                      % (ps[0]['PosterSession'],ps[0]['PosterSession'],ps[0]['Time'],ps[0]['Room']))
    f.write(sessionHeading)
    for talk in ps:
        talkCode = (u'%s-%i' % (talk['PosterSession'],talk['PositionNumber']))
        auts.append([talkCode,talk['AuthorIndex']])
        anch = (u'<a id="%s">\n' % (talk['PositionNumber']))
        details=posterHTMLAbstract(talk)
        f.write(anch+details)       
        

f.write(u'</body></html>')
f.close()
chdir('..')

In [8]:
# Building schedule page

chdir('./data')
dt = DumpTruck(dbname='Aug5_Talks.db')
ds = DumpTruck(dbname='Aug5_Sessions.db')
data = ds.dump()
chdir('..')
chdir('./Web')


fileName = "schedule.html"
f=io.open(fileName, 'w+', encoding='utf8') # hoping here

f.write(u'<html><head><title>SMPC 2019 Schedule</title><style>table, th, td {  border: 1px solid black;  border-collapse: collapse;}th, td {padding: 5px;text-align: left;}</style></head><body>\n')
f.write(u'<H1> <a id="top"></a>Presentation Schedule per day</H1>\n <br>')

f.write(u'<h2><a id="day1"></a>Monday, August 5th</h2><table style="width:100%"><caption>Event Schedule</caption>')
ps = dt.execute('SELECT * FROM `dumptruck` WHERE `SessionSlot` = ' + str(1))
f.write(u'<tr>\n')
f.write(u'<th> Times </th>\n')
for i in range(4):
    f.write((u'<th> %s </th>\n' % ps[i]['Track']))
f.write(u'</tr>')
for i in range(6):
    f.write(u'<tr>\n')
    ps = dt.execute('SELECT * FROM `dumptruck` WHERE `SessionSlot` = ' + str(i+1))
    f.write((u'<th> %s </th>\n' % ps[0]['Time']))
    for j in range(4):
        f.write((u'<th>[%s] <a href="./schedule.html#talks%s-%s">%s</a> </th>\n' % (ps[j]['SessionCode'],ps[j]['Date'][-1:],ps[j]['SessionCode'],ps[j]['SessionTitle'])))
    f.write(u'</tr>')
f.write(u'<tr><th>5:00 PM</th><th> President''s address and Keynote in Loewe Theater</th></tr>\n')
f.write(u'<tr><th>6:45 PM</th> <th> Opening Reception in Rosenthal Pavilion, Kimmel 10th floor</th></tr>\n')
f.write(u'</table>')
f.write(u'<p>')

chdir('..')
chdir('./data')
dt = DumpTruck(dbname='Aug6_Talks.db')
ds = DumpTruck(dbname='Aug6_Sessions.db')
data = ds.dump()
chdir('..')
chdir('./Web')


f.write(u'<h2><a id="day2"></a>Tuesday, August 6th</h2><table style="width:100%"><caption>Event Schedule</caption>')
ps = dt.execute('SELECT * FROM `dumptruck` WHERE `SessionSlot` = ' + str(1))
f.write(u'<tr>\n')
f.write(u'<th> Times </th>\n')
for i in range(4):
    f.write((u'<th> %s </th>\n' % ps[i]['Track']))
f.write(u'</tr>')
for i in range(6):
    f.write(u'<tr>\n')
    ps = dt.execute('SELECT * FROM `dumptruck` WHERE `SessionSlot` = ' + str(i+1))
    f.write((u'<th> %s </th>\n' % ps[0]['Time']))
    for j in range(4):
        f.write((u'<th>[%s] <a href="./schedule.html#talks%s-%s">%s</a> </th>\n' % (ps[j]['SessionCode'],ps[j]['Date'][-1:],ps[j]['SessionCode'],ps[j]['SessionTitle'])))
    f.write(u'</tr>')
f.write(u'<tr><th>3:30 PM</th><th><a href="./schedule.html#posters2-P1">Poster session P1<a> in Rosenthal Pavilion</th></tr>\n')
f.write(u'<tr><th>4:45 PM</th><th><a href="./schedule.html#posters2-P2">Poster session P2<a> in Rosenthal Pavilion</th></tr>\n')
f.write(u'<tr><th>7:00 PM</th><th>Dinner Cruise</th></tr>\n')    
f.write(u'</table>')
f.write(u'<p>')

chdir('..')
chdir('./data')
dt = DumpTruck(dbname='Aug7_Talks.db')
ds = DumpTruck(dbname='Aug7_Sessions.db')
data = ds.dump()
chdir('..')
chdir('./Web')


f.write(u'<h2><a id="day3"></a>Wednesday, August 7th</h2><table style="width:100%"><caption>Event Schedule</caption>')
ps = dt.execute('SELECT * FROM `dumptruck` WHERE `SessionSlot` = ' + str(1))
f.write(u'<tr>\n')
f.write(u'<th> Times </th>\n')
for i in range(4):
    f.write((u'<th> %s </th>\n' % ps[i]['Track']))
f.write(u'</tr>')
f.write(u'<tr>\n')
ps = dt.execute('SELECT * FROM `dumptruck` WHERE `SessionSlot` = ' + str(1))
f.write((u'<th> %s </th>\n' % ps[0]['Time']))
for j in range(4):
        f.write((u'<th>[%s] <a href="./schedule.html#talks%s-%s">%s</a> </th>\n' % (ps[j]['SessionCode'],ps[j]['Date'][-1:],ps[j]['SessionCode'],ps[j]['SessionTitle'])))
f.write(u'</tr>')
f.write(u'<tr><th>10:30 AM </th><th><a href="./schedule.html#posters3-P3">Poster session P3<a> in Rosenthal Pavilion</th></tr>\n')
f.write(u'<tr><th>11:45 AM</th><th><a href="./schedule.html#posters3-P4">Poster session P4<a> in Rosenthal Pavilion</th></tr>\n')
for i in range(2):
    f.write(u'<tr>\n')
    ps = dt.execute('SELECT * FROM `dumptruck` WHERE `SessionSlot` = ' + str(i+2))
    f.write((u'<th> %s </th>\n' % ps[0]['Time']))
    for j in range(4):
        f.write((u'<th>[%s] <a href="./schedule.html#talks%s-%s">%s</a> </th>\n' % (ps[j]['SessionCode'],ps[j]['Date'][-1:],ps[j]['SessionCode'],ps[j]['SessionTitle'])))
    f.write(u'</tr>')
f.write(u'<tr><th>5:00 PM</th><th>Business Meeting in Loewe Theater</th></tr>\n')
f.write(u'</table>')


# now to enter the more detailed components of the schedules
chdir('..')
chdir('./data')
dt = DumpTruck(dbname='Aug5_Talks.db')
ds = DumpTruck(dbname='Aug5_Sessions.db')
data = ds.dump()
chdir('..')
chdir('./Web')


f.write(u'<H1><a id="talks1"></a>Day 1 Talk Schedule </H1>\n <p>')
for i in range(len(data)):
    # specify session title, time and location
    ss = ds.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(i+1))
    sessionHeading = (u'<H3><a id="talks%s-%s"></a>Session %s, %s </H3>\n %s in %s\n <br>'
                      % (ss[0]['Date'][-1:],ss[0]['SessionCode'],ss[0]['SessionCode'],ss[0]['SessionTitle'],ss[0]['Times'],ss[0]['Room']))
    f.write(sessionHeading)
    ts = dt.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(i+1))
    f.write(u'<ul>')
    for talk in ts:
        details = talkDets(talk)
        f.write(u'<li>')
        f.write(details)
        f.write(u'</li>')
    f.write(u'</ul>')
    f.write(u'<p> Return to <a href="./schedule.html#day1">Schedule</a>. <p>')
# list talk titles (link to abstract) and authors, like to index

#f.write(u'<p> Return to <a href="./schedule.html#top">Schedule</a>. <p>')

# now to enter the later components of the schedules
chdir('..')
chdir('./data')
dt = DumpTruck(dbname='Aug6_Talks.db')
ds = DumpTruck(dbname='Aug6_Sessions.db')
data = ds.dump()
chdir('..')
chdir('./Web')


f.write(u'<H1><a id="talks2"></a>Day 2 Talk Schedule </H1>\n <p>')
#run through sessions
data = ds.dump()

for i in range(len(data)):
    # specify session title, time and location
    ss = ds.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(i+1))
    sessionHeading = (u'<H3><a id="talks%s-%s"></a>Session %s, %s </H3>\n %s in %s\n <br>'
                      % (ss[0]['Date'][-1:],ss[0]['SessionCode'],ss[0]['SessionCode'],ss[0]['SessionTitle'],ss[0]['Times'],ss[0]['Room']))
    f.write(sessionHeading)
    ts = dt.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(i+1))
    f.write(u'<ul>')
    for talk in ts:
        details = talkDets(talk)
        f.write(u'<li>')
        f.write(details)
        f.write(u'</li>')
    f.write(u'</ul>')
    f.write(u'<p> Return to <a href="./schedule.html#day2">Schedule</a>. <p>')
    
# now to enter the later components of the schedules

chdir('..')
chdir('./data')
dt = DumpTruck(dbname='Aug6_Posters.db')
data = dt.dump()
chdir('..')
chdir('./Web')


f.write(u'<H1><a id="posters2">Day 2 Poster Schedule </H1>\n <p>')

for i in ['1','2']:
    ps = dt.execute('SELECT * FROM `dumptruck` WHERE `PosterSessionNumber` = ' + i)
    sessionHeading = (u'<H2><a id="posters%s-%s"></a>Poster Session %s </H2>\n %s in %s\n <br>'
                      % (ps[0]['Date'][-1:],ps[0]['PosterSession'],ps[0]['PosterSession'],ps[0]['Time'],ps[0]['Room']))
    f.write(sessionHeading)
    f.write(u'<ul>')
    for poster in ps:
        details = posterDets(poster)
        f.write(u'<li>')
        f.write(details)
        f.write(u'</li>')
    f.write(u'</ul>')
    
    f.write(u'<p> Return to <a href="./schedule.html#day2">Schedule</a>. <p>')

    
# now to enter the later components of the schedules

chdir('..')
chdir('./data')
dt = DumpTruck(dbname='Aug7_Talks.db')
ds = DumpTruck(dbname='Aug7_Sessions.db')
data = dt.dump()
chdir('..')
chdir('./Web')


f.write(u'<H1><a id="talks3">Day 3 Talk Schedule </H1>\n <p>')
#run through sessions
data = ds.dump()
for i in range(len(data)):
    # specify session title, time and location
    ss = ds.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(i+1))
    sessionHeading = (u'<H3><a id="talks%s-%s"></a>Session %s, %s </H3>\n %s in %s\n <br>'
                      % (ss[0]['Date'][-1:],ss[0]['SessionCode'],ss[0]['SessionCode'],ss[0]['SessionTitle'],ss[0]['Times'],ss[0]['Room']))
    f.write(sessionHeading)
    ts = dt.execute('SELECT * FROM `dumptruck` WHERE `SessionNumber` = ' + str(i+1))
    f.write(u'<ul>')
    for talk in ts:
        details = talkDets(talk)
        f.write(u'<li>')
        f.write(details)
        f.write(u'</li>')
    f.write(u'</ul>')
    f.write(u'<p> Return to <a href="./schedule.html#day3">Schedule</a>. <p>')

# now to enter the later components of the schedules
chdir('..')
chdir('./data')
dt = DumpTruck(dbname='Aug7_Posters.db')
data = dt.dump()
chdir('..')
chdir('./Web')

f.write(u'<H1><a id="posters3">Day 3 Poster Schedule </H1>\n <p>')

for i in ['3','4']:
    ps = dt.execute('SELECT * FROM `dumptruck` WHERE `PosterSessionNumber` = ' + i)
    sessionHeading = (u'<H2><a id="posters%s-%s"></a>Poster Session %s </H2>\n %s in %s\n <br>'
                      % (ps[0]['Date'][-1:],ps[0]['PosterSession'],ps[0]['PosterSession'],ps[0]['Time'],ps[0]['Room']))
    f.write(sessionHeading)
    f.write(u'<ul>')
    for poster in ps:
        details = posterDets(poster)
        f.write(u'<li>')
        f.write(details)
        f.write(u'</li>')
    f.write(u'</ul>')
    
    f.write(u'<p> Return to <a href="./schedule.html#day3">Schedule</a>. <p>')

f.write(u'</body></html>')
f.close()
chdir('..')

In [9]:
pwd

u'/Users/Finn/Documents/2018-19/SMPC/Documents/Presentations_sharable'

In [10]:
# now to sort out the full authors list and generate index

# first get all the index strings and alphabetise
# then run through auts and append the codes

UniqueAuthors = set()
for talk in auts:
    for ai in talk[1]:
        if ai not in UniqueAuthors:
            UniqueAuthors.add(ai)
            #print ai
#       
print len(UniqueAuthors)

AuInd = []
for ai in UniqueAuthors:
    AuInd.append([ai,[]])
    
# order set
    
for work in auts:
    for ai in work[1]:
        for a in AuInd:
            if a[0]==ai:
                a[1].append(work[0])

# then go through it all one last time to generate index list


685


In [11]:
chdir('./Web')

fileName = "author_index.html"
f=io.open(fileName, 'w+', encoding='utf8') # hoping here

f.write(u'<html><head><title>Author Index</title></head><body>\n')
f.write(u'<H1> Author Index</H1>\n <br>')
        
AuInd.sort()
for a in AuInd:
    entry = u''
    anchor = urllib.quote(a[0].encode('ascii',errors='ignore'))
    entry += '<a id="' + anchor + '"></a>'
    entry += a[0]
    for preCode in a[1]:
        bits = preCode.split('-')
        entry += ' <a href="./abstracts.html#' + preCode + '">' + preCode + '</a>'
    entry += '\n <br>'
    f.write(entry)

f.write(u'<p> Return to <a href="./schedule.html#top">Schedule</a>. <p>')

f.write(u'</body></html>')
f.close()
chdir('..')